<a href="https://colab.research.google.com/github/ashaaher/Deep-Learning/blob/master/HW1_MNIST_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


import keras
import pandas as pd
import numpy as np,sys
from keras.datasets import mnist
import copy
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
import os
np.random.seed(1)

In [1]:
import keras
import pandas as pd
import numpy as np,sys
from keras.datasets import mnist
import copy
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
import os
np.random.seed(1)

Using TensorFlow backend.


# Loading and Visualization of Dataset

In [2]:
from keras.datasets import mnist
(x_train_label,y_train_label),(x_test_label,y_test_label)=mnist.load_data()
for i in range(0, 9):
	pyplot.subplot(330 + 1 + i)
	pyplot.imshow(x_train[i], cmap=pyplot.get_cmap('gray'))

11493376/11490434 [==============================] - 0s 0us/step


# Normalizing the inputs

In [0]:
images,labels = (x_train_label.reshape(len(x_train_label), 28*28)/255,y_train_label)

# One hot encoding

In [0]:
one_hot_labels = np.zeros((len(labels),10))

In [0]:
for i, l in enumerate(labels):
  one_hot_labels[i][l] = 1
labels =one_hot_labels

In [0]:
test_images = x_test_label.reshape(len(x_test_label),28*28)/255
test_labels = np.zeros((len(y_test_label),10))
for i,l in enumerate(y_test_label):
    test_labels[i][l] =1

# Image augmentation

In [0]:
x_train_label = x_train_label.reshape((x_train_label.shape[0], 28, 28, 1))
x_test_label = x_test_label.reshape((x_test_label.shape[0], 28, 28, 1))
# convert from int to float
x_train_label = x_train_label.astype('float32')
x_test_label = x_test_label.astype('float32')
# define data preparation
datagen = ImageDataGenerator()
# fit parameters from data
datagen.fit(x_train_label)
# configure batch size and retrieve one batch of images
os.makedirs('images2')

# Using 3 Neuron Layers (Input and Hidden layers)

In [0]:
np.random.seed(1)
batch_size = 100
relu = lambda x:(x>=0)*x
relu2deriv = lambda x:x >=0
def softmax(x):
   temp = np.exp(x)
   return temp/np.sum(temp,axis =1,keepdims =True)
alpha,iterations,hidden_size, pixel_per_image,num_labels =(0.01,300,400,784,10)
weights_0_1 = np.random.randn(pixel_per_image,hidden_size) * np.sqrt(2.0/hidden_size)
weights_1_2 =np.random.randn(hidden_size,400) * np.sqrt(2.0/400)
weights_2_3 =np.random.randn(400,num_labels) * np.sqrt(2.0/num_labels)

# Implementing Minibatch gradient descent along with appropriate learning rate and various dropout rates.

In [0]:

for j in range (iterations):
  correct_cnt = 0
  error =0.0
  for i in range(int(len(images)/batch_size)):
    batch_start,batch_end = ((i*batch_size),((i+1)*batch_size))
    layer_0=images[batch_start:batch_end]
    layer_1=relu(np.dot(layer_0,weights_0_1))
    dropout_mask = np.random.randint(2,size=layer_1.shape)
    layer_1 *= dropout_mask * 2
    layer_2=relu(np.dot(layer_1,weights_1_2))
    dropout_mask = np.random.randint(2,size=layer_2.shape)
    layer_2 *= dropout_mask * 2
    layer_3=softmax(np.dot(layer_2,weights_2_3))
    error+=np.sum((labels[batch_start:batch_end]-layer_3)** 2)

    for k in range (batch_size):
      correct_cnt += int(np.argmax(layer_3[k:k+1])==np.argmax(labels[batch_start+k:batch_start+k+1]))
    layer_3_delta = (labels[batch_start:batch_end]-layer_3) / (batch_size * layer_3.shape[0])
    layer_2_delta = layer_3_delta.dot(weights_2_3.T)*relu2deriv(layer_2)
    layer_2_delta *= dropout_mask
    layer_1_delta = layer_2_delta.dot(weights_1_2.T)*relu2deriv(layer_1)
    layer_1_delta *= dropout_mask

    weights_2_3 += alpha * layer_2.T.dot(layer_3_delta)
    weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
    weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)
test_correct_cnt = 0

for i in range (len(test_images)):
  layer_0 = test_images[i:i+1]
  layer_1 = relu(np.dot(layer_0,weights_0_1))
  layer_2 = relu(np.dot(layer_1,weights_1_2))
  layer_3 = np.dot(layer_2,weights_2_3)
  test_correct_cnt += int(np.argmax(layer_2)==np.argmax(test_labels[i:i+1]))

if(j % 10 ==0):
  sys.stdout.write("\n"+ "I:" + str(j) + "Test-Acc:" + str(test_correct_cnt/float(len(test_images))) + " Train-Acc:" + str(correct_cnt/float(len(images))))



# confusion matrix

In [0]:
[x_train_label,y_train_label] = digitTrain4DArrayData;
whos y_train_label

In [0]:
layers = [
    imageInputLayer([28 28 1])
    
    NN2dLayer(3,8,'Padding','same')
    batchNormalizationLayer
    reluLayer    
    NN2dLayer(3,16,'Padding','same','Stride',2)
    batchNormalizationLayer
    reluLayer
    NN2dLayer(3,32,'Padding','same','Stride',2)
    batchNormalizationLayer
    reluLayer
    
    fullyConnectedLayer(10)
    softmaxLayer
    classificationLayer];

In [0]:
options = trainingOptions('sgdm',...
    'MaxEpochs',5,...
    'Verbose',false,...
    'Plots','training-progress');
net = trainNetwork(x_train_label,y_train_label,layers,options);

In [0]:
[x_test_label,y_test_label] = digitTest4DArrayData;
y_Predicted = classify(net,x_test_label);

In [0]:
plotconfusion(y_test_label,y_Predicted)